In [5]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
  list_strings = []
  for index in range(len(df)):
    if index % 1000 == 0:
      print(str(index)+' / '+str(len(df)))
    text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
    string = -1
    if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
        string = df.loc[index,"extended_tweet"]
    if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
        string = df.loc[index,"retweeted_status"]
    if type(string) == str :
        if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
          s = re.search('full_text\':(.+?)https',string).group(1)
        if(re.search('text\':(.+?)https',string)!= None):
          s = re.search('text\':(.+?)https',string).group(1)
        else: 
          continue
        list_strings.append(s)
        #print(s)         
    else:
      list_strings.append(text)
      #print(text)
      

  return list_strings

In [7]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [364]:
period = '_JanFeb2020'  # '', '_JanFeb2020', '_MarchApril2021', '_SeptOct2020'

In [365]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

In [366]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  4876
USA:  7519
China&USA:  12395


In [367]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

0 / 4876
1000 / 4876
2000 / 4876
3000 / 4876
4000 / 4876
0 / 7519
1000 / 7519
2000 / 7519
3000 / 7519
4000 / 7519
5000 / 7519
6000 / 7519
7000 / 7519
0 / 12395
1000 / 12395
2000 / 12395
3000 / 12395
4000 / 12395
5000 / 12395
6000 / 12395
7000 / 12395
8000 / 12395
9000 / 12395
10000 / 12395
11000 / 12395
12000 / 12395


In [368]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['talk university challenge china face deal misconception people medium watch', 'spring always bring along lively saturate sea colorful bloom flower look collection flower blossom across china enjoy upcoming new season', 'mean carnival brazil year one samba school take inspiration china', 'china world make great stride track infectious disease take unexpected turn make hard track', 'fake news story case covid lead protest evacuation people', 'transmission outside china graphic', 'world health organization announce lead team international expert currently china', 'outside china confirm case novel covid half cruise ship dock japan rest scatter among country mostly asia', 'five cargo aircraft airway head city beijing shanghai carry ton medical supply gratis include million mask hand send epidemic prevention supply purchase china government', 'case miss jinping chair lead body meeting covid control']

USA:
['shameful attack bus carry evacuee china coronavirus outbreak', 'china hubei

In [369]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  3617
USA:  3380
China&USA:  4883


In [370]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>200])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>400])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>600])

China
['china', 'novel', 'hospital', 'coronavirus', 'case', 'outbreak', 'new', 'patient', 'fight', 'say', 'confirm', 'people', 'epidemic', 'medical', 'hubei', 'province', 'battle', 'support', 'report', 'health', 'live', 'city', 'amid', 'death', 'day', 'control', 'virus', 'year', 'take', 'effort', 'country', 'help', 'one', 'discharge', 'work']

USA
['coronavirus', 'china', 'outbreak', 'virus', 'case', 'say', 'new', 'spread', 'death', 'people', 'health', 'late', 'report', 'cruise', 'first', 'ship']

China&USA
['china', 'coronavirus', 'outbreak', 'case', 'new', 'novel', 'say', 'virus', 'hospital', 'people', 'death', 'spread', 'health', 'patient', 'confirm', 'report', 'fight', 'city', 'late']


# less frequent words

In [371]:
# less frequent words:
print('Less frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]<10]))
print('More frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]>=10]))
less_frequent_words_China = [key for key in freq_dict_China.keys() if freq_dict_China[key]<10]

print('Less frequent USA: ', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]<10]))
print('More frequent USA:', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>=10]))
less_frequent_words_USA = [key for key in freq_dict_USA.keys() if freq_dict_USA[key]<10]

print('Less frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<10]))
print('More frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>=10]))
less_frequent_words_China_USA = [key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<10]

Less frequent China:  2702
More frequent China:  915
Less frequent USA:  2300
More frequent USA: 1080
Less frequent China&USA:  3291
More frequent China&USA:  1592


In [372]:
# discard less frequent words
def clean_words(cleaned_text, unuseful_words):
  re_cleaned_text = cleaned_text.copy()
  for txt in range(len(re_cleaned_text)):
    if txt % 2000 == 0:
      print(txt, '/',len(re_cleaned_text))
    w = re_cleaned_text[txt].split()
    for word in unuseful_words:
      while word in w:
        w.remove(word)
    re_cleaned_text[txt] = ' '.join(w)
  return re_cleaned_text

In [373]:
cleaned_mostfreq_text_China = clean_words(cleaned_text_China, less_frequent_words_China)
cleaned_mostfreq_text_USA = clean_words(cleaned_text_USA, less_frequent_words_USA)
cleaned_mostfreq_text_China_USA = clean_words(cleaned_text_China_USA, less_frequent_words_China_USA)

0 / 4843
2000 / 4843
4000 / 4843
0 / 7519
2000 / 7519
4000 / 7519
6000 / 7519
0 / 12362
2000 / 12362
4000 / 12362
6000 / 12362
8000 / 12362
10000 / 12362
12000 / 12362


In [374]:
freq_dict_China = frequency_dictionary(cleaned_mostfreq_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostfreq_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostfreq_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  915
USA:  1080
China&USA:  1592


# Build Network

In [375]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   3018
1         novel   1385
2      hospital    969
3   coronavirus    949
4          case    863
5      outbreak    846
6           new    657
7       patient    651
8         fight    583
9           say    521
10      confirm    454
11       people    426
12     epidemic    403
13      medical    378
14        hubei    369
15     province    355
16       battle    311
17      support    310
18       report    299
19       health    294
20         live    294
21         city    293
22         amid    289
23        death    288
24          day    273
25      control    266
26        virus    263
27         year    233
28         take    223
29       effort    213

USA
           Word  Count
0   coronavirus   5135
1         china   3946
2      outbreak   1404
3         virus   1132
4          case   1110
5           say    933
6           new    906
7        spread    858
8         death    734
9        people    624
10       health    603


In [376]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [377]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [378]:
network_China, network_df_China = create_network(cleaned_mostfreq_text_China)
network_USA, network_df_USA = create_network(cleaned_mostfreq_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostfreq_text_China_USA)

In [379]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                         weight
(china, novel)             1024
(china, outbreak)           644
(china, coronavirus)        619
(confirm, case)             571
(patient, hospital)         560
(china, case)               527
(china, fight)              502
(novel, coronavirus)        483
(new, case)                 479
(china, new)                427
(china, say)                422
(novel, outbreak)           394
(case, report)              368
(case, death)               357
(epidemic, china)           346
(china, support)            339
(china, hospital)           334
(china, confirm)            311
(new, confirm)              296
(china, people)             281
(china, province)           272
(china, battle)             266
(china, hubei)              258
(report, new)               245
(new, death)                245
(makeshift, hospital)       233
(fight, novel)              233
(china, effort)             232
(novel, hospital)           231
(coronavirus, outbreak)     227



#Graph


In [380]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [381]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [382]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  915
Edges:  60917
Is connected:  True

USA:
Nodes:  1080
Edges:  61951
Is connected:  True

China&USA:
Nodes:  1592
Edges:  131038
Is connected:  True


# PageRank

In [383]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [209]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [210]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [384]:
thr = 0.0004
print('China: ', len(threshold(pr_China,thr)))
print()
print(threshold(pr_China,thr).iloc[:30])
print()
print('USA: ', len(threshold(pr_USA,thr)))
print()
print(threshold(pr_USA,thr).iloc[:30])
print()
print('China&USA: ', len(threshold(pr_China_USA,thr)))
print()
print(threshold(pr_China_USA,thr).iloc[:30])

China:  516

              0         1
0         china  0.056357
1         novel  0.027831
2      hospital  0.019156
3          case  0.017943
4   coronavirus  0.017606
5      outbreak  0.016558
6           new  0.013158
7       patient  0.013123
8           say  0.011960
9         fight  0.011002
10      confirm  0.010191
11       people  0.009482
12      medical  0.008473
13     province  0.008417
14        hubei  0.008305
15     epidemic  0.008215
16       health  0.006991
17         city  0.006711
18       report  0.006655
19        death  0.006641
20      support  0.006538
21       battle  0.006012
22          day  0.005957
23        virus  0.005790
24          one  0.005378
25         year  0.005230
26      control  0.005179
27         amid  0.005125
28      country  0.004860
29         take  0.004819

USA:  509

              0         1
0   coronavirus  0.062184
1         china  0.050582
2      outbreak  0.021386
3          case  0.016280
4         virus  0.014761
5           s

# TF-IDF

In [145]:
tfidf = TfidfVectorizer(ngram_range=(1,1))   # ngram range can be changed to obtain measures regarding n grams instead of single words

X_China = tfidf.fit_transform(cleaned_mostfreq_text_China).toarray()    # entry (i,j) if Tfidf measure of word_list[j] in document i
word_list_China = tfidf.get_feature_names_out()

X_USA = tfidf.fit_transform(cleaned_mostfreq_text_USA).toarray()
word_list_USA = tfidf.get_feature_names_out()

X_China_USA = tfidf.fit_transform(cleaned_mostfreq_text_China_USA).toarray()
word_list_China_USA = tfidf.get_feature_names_out()


In [146]:
tfidf_df_China = pd.DataFrame(X_China,columns = word_list_China)

tfidf_df_USA = pd.DataFrame(X_USA,columns = word_list_USA)

tfidf_df_China_USA = pd.DataFrame(X_China_USA,columns = word_list_China_USA)

In [147]:
tfidf_word_measure_China = np.mean(tfidf_df_China,axis = 0)
tfidf_word_measure_China = tfidf_word_measure_China.sort_values(ascending = False)
tfidf_word_measure_USA = np.mean(tfidf_df_USA,axis = 0)
tfidf_word_measure_USA = tfidf_word_measure_USA.sort_values(ascending = False)
tfidf_word_measure_China_USA = np.mean(tfidf_df_China_USA,axis = 0)
tfidf_word_measure_China_USA = tfidf_word_measure_China_USA.sort_values(ascending = False)

In [148]:
print('China:')
print(tfidf_word_measure_China[0:30])
print()
print('USA:')
print(tfidf_word_measure_USA[0:30])
print()
print('China&USA:')
print(tfidf_word_measure_China_USA[0:30])
print()

China:
china          0.057308
covid          0.032324
case           0.029068
novel          0.027613
coronavirus    0.027073
new            0.025310
vaccine        0.022850
hospital       0.022707
say            0.020790
outbreak       0.019451
report         0.017290
fight          0.016928
patient        0.016614
people         0.015870
confirm        0.014493
watch          0.014340
health         0.014284
country        0.013984
day            0.013548
live           0.013418
province       0.012843
first          0.012796
city           0.012772
death          0.012311
year           0.012188
epidemic       0.011602
world          0.011589
amid           0.011068
medical        0.011029
president      0.010939
dtype: float64

USA:
coronavirus    0.049964
covid          0.048798
china          0.041922
vaccine        0.031179
case           0.028552
say            0.027605
new            0.026548
report         0.017858
death          0.017717
outbreak       0.017472
virus       

# reduced graph

In [385]:
# less important words:
less_important_words_China = list(threshold_reverse(pr_China,thr)[0])

less_important_words_USA = list(threshold_reverse(pr_USA,thr)[0])

less_important_words_China_USA = list(threshold_reverse(pr_China_USA,thr)[0])

In [386]:
cleaned_mostimp_text_China = clean_words(cleaned_mostfreq_text_China,less_important_words_China)
cleaned_mostimp_text_USA = clean_words(cleaned_mostfreq_text_USA,less_important_words_USA)
cleaned_mostimp_text_China_USA = clean_words(cleaned_mostfreq_text_China_USA,less_important_words_China_USA)

0 / 4843
2000 / 4843
4000 / 4843
0 / 7519
2000 / 7519
4000 / 7519
6000 / 7519
0 / 12362
2000 / 12362
4000 / 12362
6000 / 12362
8000 / 12362
10000 / 12362
12000 / 12362


In [387]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  516
USA:  509
China&USA:  490


In [388]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   3018
1         novel   1385
2      hospital    969
3   coronavirus    949
4          case    863
5      outbreak    846
6           new    657
7       patient    651
8         fight    583
9           say    521
10      confirm    454
11       people    426
12     epidemic    403
13      medical    378
14        hubei    369
15     province    355
16       battle    311
17      support    310
18       report    299
19       health    294
20         live    294
21         city    293
22         amid    289
23        death    288
24          day    273
25      control    266
26        virus    263
27         year    233
28         take    223
29       effort    213

USA
           Word  Count
0   coronavirus   5135
1         china   3946
2      outbreak   1404
3         virus   1132
4          case   1110
5           say    933
6           new    906
7        spread    858
8         death    734
9        people    624
10       health    603


In [389]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [390]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [391]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                         weight
(china, novel)             1024
(china, outbreak)           644
(china, coronavirus)        619
(confirm, case)             571
(patient, hospital)         560
(china, case)               527
(china, fight)              502
(novel, coronavirus)        483
(new, case)                 479
(china, new)                427
(china, say)                422
(novel, outbreak)           394
(case, report)              368
(case, death)               357
(epidemic, china)           346
(china, support)            339
(china, hospital)           334
(china, confirm)            311
(new, confirm)              296
(china, people)             281
(china, province)           272
(china, battle)             266
(china, hubei)              258
(report, new)               245
(new, death)                245
(makeshift, hospital)       233
(fight, novel)              233
(china, effort)             232
(novel, hospital)           231
(coronavirus, outbreak)     227



In [392]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [393]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [394]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  516
Edges:  37525
Is connected:  True

USA:
Nodes:  509
Edges:  34977
Is connected:  True

China&USA:
Nodes:  490
Edges:  50030
Is connected:  True


#Save edge list

In [395]:
filename = './edgelist_China_JanFeb2020.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_JanFeb2020.csv
files.download("edgelist_China_JanFeb2020.csv")

filename = './edgelist_USA_JanFeb2020.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA_JanFeb2020.csv
files.download("edgelist_USA_JanFeb2020.csv")

filename = './edgelist_China_USA_JanFeb2020.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA_JanFeb2020.csv
files.download("edgelist_China_USA_JanFeb2020.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [361]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [396]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,"China_JanFeb2020")
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,"USA_JanFeb2020")
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,"China_USA_JanFeb2020")

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
            Id        Label
0        china        china
1        novel        novel
2     hospital     hospital
3  coronavirus  coronavirus
4         case         case

USA:
            Id        Label
0  coronavirus  coronavirus
1        china        china
2     outbreak     outbreak
3        virus        virus
4         case         case

China&USA:
            Id        Label
0        china        china
1  coronavirus  coronavirus
2     outbreak     outbreak
3         case         case
4          new          new

